In [6]:
import uuid
import itertools
import numpy as np
import pandas as pd
import tensorflow as tf
import json
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix

In [24]:
!gsutil cp gs://financial_fraud_detection/fraud_data_kaggle.csv .
data = pd.read_csv('fraud_data_kaggle.csv')
data.head()  

Copying gs://financial_fraud_detection/fraud_data_kaggle.csv...
/ [1 files][470.7 MiB/470.7 MiB]                                                
Operation completed over 1 objects/470.7 MiB.                                    


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [25]:
fraud = data[data['isFraud'] == 1]
not_fraud = data[data['isFraud'] == 0]
# Take a random sample of non fraud rows
not_fraud_sample = not_fraud.sample(random_state=2, frac=.005)

# Put it back together and shuffle
df = pd.concat([not_fraud_sample,fraud])
df = shuffle(df, random_state=2)

# Remove a few columns (isFraud is the label column we'll use, not isFlaggedFraud)
df = df.drop(columns=['nameOrig', 'nameDest', 'isFlaggedFraud'])
# Adding transaction id to identify back after prediction
df['transactionId'] = [str(uuid.uuid4()) for _ in range(len(df.index))]
# Preview the updated dataset
df.head()
train_test_split = int(len(df) * .8)

train_set = df[:train_test_split]
test_set = df[train_test_split:]

train_labels = train_set.pop('isFraud')
test_labels = test_set.pop('isFraud')

In [8]:
fc = tf.feature_column
CATEGORICAL_COLUMNS = ['type']
NUMERIC_COLUMNS = ['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
KEY_COLUMN = 'transactionId'
def one_hot_cat_column(feature_name, vocab):
    return tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocab))

feature_columns = []

for feature_name in CATEGORICAL_COLUMNS:
    vocabulary = train_set[feature_name].unique()
    feature_columns.append(one_hot_cat_column(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name,
                                           dtype=tf.float32))


In [9]:
NUM_EXAMPLES = len(train_labels)
def make_input_fn(X, y, n_epochs=None, shuffle=True):
  def input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((dict(X), y))
    if shuffle:
      dataset = dataset.shuffle(NUM_EXAMPLES)
    dataset = dataset.repeat(n_epochs)
    dataset = dataset.batch(NUM_EXAMPLES)
    return dataset
  return input_fn

# Define training and evaluation input functions
train_input_fn = make_input_fn(train_set, train_labels)
eval_input_fn = make_input_fn(test_set, test_labels, shuffle=False, n_epochs=1)

In [10]:
n_batches = 1
model = tf.estimator.BoostedTreesClassifier(feature_columns,
                                          n_batches_per_layer=n_batches)
model = tf.contrib.estimator.forward_features(model,KEY_COLUMN)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpgxzip7p8', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1a7b996910>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
The old _FeatureColumn APIs a

In [11]:
model.train(train_input_fn, max_steps=100)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
I

In [12]:
result = model.evaluate(eval_input_fn)
print(pd.Series(result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-18T12:55:17Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpgxzip7p8/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-05-18-12:55:19
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.99374765, accuracy_baseline = 0.8006753, auc = 0.99866736, auc_precision_recall = 0.9978666, average_loss = 0.025304431, global_step = 100, label/mean = 0.19932474, loss = 0.025304431, precision = 0.9759556, prediction/mean = 0.20034805, recall = 0.9930991
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Re

In [13]:
pred_dicts = list(model.predict(eval_input_fn))
probabilities = pd.Series([pred['logistic'][0] for pred in pred_dicts])

for i,val in enumerate(probabilities[:30]):
  print('Predicted: ', round(val), 'Actual: ', test_labels.iloc[i])
  print()

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpgxzip7p8/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predicted:  0 Actual:  0

Predicted:  1 Actual:  1

Predicted:  0 Actual:  0

Predicted:  0 Actual:  0

Predicted:  0 Actual:  0

Predicted:  0 Actual:  0

Predicted:  0 Actual:  0

Predicted:  0 Actual:  0

Predicted:  0 Actual:  0

Predicted:  0 Actual:  0

Predicted:  0 Actual:  0

Predicted:  0 Actual:  0

Predicted:  0 Actual:  0

Predicted:  0 Actual:  0

Predicted:  0 Actual:  0

Predicted:  0 Actual:  0

Predicted:  0 Actual:  0

Predicted:  0 Actual:  0

Predicted:  0 Actual:  0

Predicted:  1 Actual:  1

Predicted:  0 Actual:  0

Predicted:  0 Actual:  0

Predicted:  0 Actual:  0

Predicted:  0 Actual:  0

Predicted:  1 Actual:  1

Predicted:

In [14]:
GCP_PROJECT = 'next-demo-2020'
MODEL_BUCKET = 'gs://credit-card-fraud-detection-tf-model'

In [15]:
def json_serving_input_fn():
    feature_placeholders = {
        'type': tf.placeholder(tf.string, [None]),
        'step': tf.placeholder(tf.float32, [None]),
        'amount': tf.placeholder(tf.float32, [None]),
        'oldbalanceOrg': tf.placeholder(tf.float32, [None]),
        'newbalanceOrig': tf.placeholder(tf.float32, [None]),
        'oldbalanceDest': tf.placeholder(tf.float32, [None]),
        'newbalanceDest': tf.placeholder(tf.float32, [None]),
         KEY_COLUMN: tf.placeholder_with_default(tf.constant(['nokey']), [None])
    }
    features = {key: tf.expand_dims(tensor, -1)
                for key, tensor in feature_placeholders.items()}
    return tf.estimator.export.ServingInputReceiver(features,feature_placeholders)
export_path = model.export_saved_model(
    MODEL_BUCKET + '/explanations-with-key',
    serving_input_receiver_fn=json_serving_input_fn
).decode('utf-8')

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Signatures EXCLUDED from export because they cannot be be served via TensorFlow Serving APIs:
INFO:tensorflow:'serving_default' : Classification input must be a single string Tensor; got {'type': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=string>, 'step': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=float32>, 'a

In [16]:
!saved_model_cli show --dir $export_path --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['predict']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['amount'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder_2:0
    inputs['newbalanceDest'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder_6:0
    inputs['newbalanceOrig'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder_4:0
    inputs['oldbalanceDest'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder_5:0
    inputs['oldbalanceOrg'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder_3:0
    inputs['step'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder_1:0
    inputs['transactionId'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: PlaceholderWithDefault:0
    inputs['type'] tensor_in

In [21]:
MODEL = 'fraud_detection_with_key'
VERSION = 'v1'

In [84]:
!gcloud ai-platform models create $MODEL
!gcloud beta ai-platform versions create $VERSION \
--model $MODEL \
--origin $export_path \
--runtime-version 1.15 \
--framework TENSORFLOW \
--python-version 3.7 \
--machine-type n1-standard-4 \
--num-paths 10

Creating version (this might take a few minutes)......done.                    


In [85]:
!gcloud ai-platform versions describe $VERSION --model $MODEL

createTime: '2020-05-18T01:47:57Z'
deploymentUri: gs://credit-card-fraud-detection-tf-model/explanations-with-key/1589766342
etag: ciscgfdz0xM=
framework: TENSORFLOW
isDefault: true
machineType: n1-standard-4
name: projects/next-demo-2020/models/fraud_detection_with_key/versions/v1
pythonVersion: '3.7'
runtimeVersion: '1.15'
state: READY


In [17]:
fraud_indices = []

for i,val in enumerate(test_labels):
    if val == 1:
        fraud_indices.append(i)

In [18]:
num_test_examples = 10
import numpy as np 

def convert(o):
    if isinstance(o, np.generic): return o.item()  
    raise TypeError

for i in range(num_test_examples):
    test_json = {}
    ex = test_set.iloc[fraud_indices[i]]
    keys = ex.keys().tolist()
    vals = ex.values.tolist()
    for idx in range(len(keys)):
        test_json[keys[idx]] = vals[idx]

    print(test_json)
    with open('data.txt', 'a') as outfile:
        json.dump(test_json, outfile, default=convert)
        outfile.write('\n')

{'step': 476, 'type': 'TRANSFER', 'amount': 1048.63, 'oldbalanceOrg': 1048.63, 'newbalanceOrig': 0.0, 'oldbalanceDest': 0.0, 'newbalanceDest': 0.0, 'transactionId': '1eee7190-7eb4-4af2-89b4-fee5e6a9e33f'}
{'step': 390, 'type': 'TRANSFER', 'amount': 638693.49, 'oldbalanceOrg': 638693.49, 'newbalanceOrig': 0.0, 'oldbalanceDest': 0.0, 'newbalanceDest': 0.0, 'transactionId': 'dc417f1b-6224-4255-8898-a69044096f02'}
{'step': 355, 'type': 'CASH_OUT', 'amount': 5338162.8, 'oldbalanceOrg': 5338162.8, 'newbalanceOrig': 0.0, 'oldbalanceDest': 181895.58, 'newbalanceDest': 5520058.37, 'transactionId': 'e46d8228-97e7-467b-8411-ca83ae5bb0d0'}
{'step': 356, 'type': 'TRANSFER', 'amount': 357226.8, 'oldbalanceOrg': 357226.8, 'newbalanceOrig': 0.0, 'oldbalanceDest': 0.0, 'newbalanceDest': 0.0, 'transactionId': '5275f001-cbc6-41a9-9903-3b335c00167d'}
{'step': 345, 'type': 'TRANSFER', 'amount': 128936.95, 'oldbalanceOrg': 128936.95, 'newbalanceOrig': 0.0, 'oldbalanceDest': 0.0, 'newbalanceDest': 0.0, 'tran

In [23]:
!cat data.txt

{"step": 476, "type": "TRANSFER", "amount": 1048.63, "oldbalanceOrg": 1048.63, "newbalanceOrig": 0.0, "oldbalanceDest": 0.0, "newbalanceDest": 0.0, "transactionId": "0"}
{"step": 476, "type": "TRANSFER", "amount": 1048.63, "oldbalanceOrg": 1048.63, "newbalanceOrig": 0.0, "oldbalanceDest": 0.0, "newbalanceDest": 0.0,"transactionId": "1111"}
{"step": 476, "type": "TRANSFER", "amount": 1048.63, "oldbalanceOrg": 1048.63, "newbalanceOrig": 1048.63, "oldbalanceDest": 0.0, "newbalanceDest": 1048.63,"transactionId": "1111"}
{"step": 476, "type": "TRANSFER", "amount": 1048.63, "oldbalanceOrg": 1048.63, "newbalanceOrig": 0.0, "oldbalanceDest": 0.0, "newbalanceDest": 0.0, "transactionId": "1eee7190-7eb4-4af2-89b4-fee5e6a9e33f"}
{"step": 390, "type": "TRANSFER", "amount": 638693.49, "oldbalanceOrg": 638693.49, "newbalanceOrig": 0.0, "oldbalanceDest": 0.0, "newbalanceDest": 0.0, "transactionId": "dc417f1b-6224-4255-8898-a69044096f02"}
{"step": 355, "type": "CASH_OUT", "amount": 5338162.8, "oldbalan

In [22]:
!gcloud ai-platform predict --model $MODEL \
--version $VERSION \
--json-instances='data.txt' \
--signature-name='predict'

ALL_CLASS_IDS  ALL_CLASSES   CLASS_IDS  CLASSES  LOGISTIC        LOGITS         PROBABILITIES                  TRANSACTION_ID
[0, 1]         [u'0', u'1']  [1]        [u'1']   [0.988502502]   [4.45405817]   [0.0114975385, 0.988502502]    [u'0']
[0, 1]         [u'0', u'1']  [1]        [u'1']   [0.988502502]   [4.45405817]   [0.0114975385, 0.988502502]    [u'1111']
[0, 1]         [u'0', u'1']  [0]        [u'0']   [0.0222329199]  [-3.78369617]  [0.97776711, 0.0222329479]     [u'1111']
[0, 1]         [u'0', u'1']  [1]        [u'1']   [0.988502502]   [4.45405817]   [0.0114975385, 0.988502502]    [u'1eee7190-7eb4-4af2-89b4-fee5e6a9e33f']
[0, 1]         [u'0', u'1']  [1]        [u'1']   [0.997712731]   [6.07810307]   [0.00228727772, 0.997712731]   [u'dc417f1b-6224-4255-8898-a69044096f02']
[0, 1]         [u'0', u'1']  [1]        [u'1']   [0.993289351]   [4.99732447]   [0.00671066204, 0.993289411]   [u'e46d8228-97e7-467b-8411-ca83ae5bb0d0']
[0, 1]         [u'0', u'1']  [1]        [u'1']   [0.996